In [2]:
import tweepy
import emoji

import regex as re
import numpy as np
import pandas as pd

from google.cloud import translate

In [3]:
# Set authorization keys
auth = tweepy.OAuthHandler('XXX', 'XXX')
auth.set_access_token("XX-XX", 
    "XXX")

api = tweepy.API(auth, wait_on_rate_limit=True)

In [10]:
M5S_NAME = '@Mov5Stelle'
LEGA_NAME = '@LegaSalvini'
PADE_NAME = '@pdnetwork'
FORZA_NAME ='@forza_italia'

party_names = [M5S_NAME, LEGA_NAME, PADE_NAME, FORZA_NAME]

sample_user_id = 2149131133

translate_client = translate.Client()

In [6]:
def get_followers(name='@pdnetwork', size=1):
    ids = []
    stop = 0
    for page in tweepy.Cursor(api.followers_ids, screen_name="@pdnetwork").pages():
        if stop>size:
            break

        stop+=1
        ids.extend(page)
    return ids

In [7]:
def get_tweets_from_user(user_id):
    try:
        tweets = api.user_timeline(user_id, per_page=50)
    except:
        return None
    
    if len(tweets) < 10:
        return None
    
    tweets_info = []
    # relevant info from tweet (coordinates, place and geo removed due to null values)
    tweet_info_needed = ['id_str', 'created_at', 'text',
                         'is_quote_status', 'retweet_count', 'lang'
                        ]
    user_info_needed = ['name', 'location', 'description', 'id_str']
    
    for tweet in tweets:
        to_append = {}
        for info in tweet_info_needed:
            to_append[info] = tweet._json[info]
        
        hashtags = tweet._json['entities']['hashtags']
        
        to_append['hashtags'] = []
        for hashtag in hashtags:
            to_append['hashtags'].append(hashtag['text'])
        
        for info in user_info_needed:
            to_append['user_' + info] = tweet._json['user'][info]
        
        tweets_info.append(to_append)
        
    return tweets_info

In [8]:
def get_emojis(text):
    emoji_list = []
    data = re.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [9]:
def del_emojis(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [16]:
def translate_tweet(tweet_text, lang='en'):
    return translate_client.translate(tweet_text, target_language=lang)['translatedText']

In [22]:
def clean_tweet(tweet):
    cleaned_tweet = re.sub('@\S+', '', tweet) # Remove mentions
    cleaned_tweet = re.sub('https\S+', '', cleaned_tweet) # Remove urls
    cleaned_tweet = re.sub('\S+…', '', cleaned_tweet) # Remove truncated last word
    cleaned_tweet = re.sub('#', '', cleaned_tweet)
    cleaned_tweet = re.sub('\n', '', cleaned_tweet)
    cleaned_tweet = re.sub('\t', '', cleaned_tweet)
    
    return cleaned_tweet.strip()

In [10]:
def get_tweets_from_party_followers(party):
    df = pd.DataFrame()
    user_ids = get_followers(party)
    
    for user_id in user_ids:
        tweets = get_tweets_from_user(user_id)
        
        if tweets == None:
            continue
        
        if not is_user_valid(tweets):
            continue
        
        for tweet in tweets:
            df = df.append(tweet)
        
    return df

In [18]:
PADE_ids = get_followers()

In [32]:
PADE_ids = np.array(PADE_ids)
np.save(PADE_NAME+'.npy', PADE_ids)

In [34]:
np.load(PADE_NAME+'.npy')[0]

758072107981017089

In [20]:
df = pd.DataFrame()
user_ids = PADE_ids[:10]

empty_users = 0

for user_id in user_ids:
    tweets = get_tweets_from_user(user_id)
        
    if tweets == None:
        empty_users+=1
        continue
    print(f'getting tweets for user with user_id: {user_id}')
    for tweet in tweets:
        df = df.append(tweet, ignore_index=True)

getting tweets for user with user_id: 758072107981017089
getting tweets for user with user_id: 1167844224936685573
getting tweets for user with user_id: 492216260
getting tweets for user with user_id: 994918270041055233


In [24]:
df['clean_text'] = df['text'].apply(clean_tweet)
df['emojis'] = df['text'].apply(get_emojis)

In [27]:
df['translated_text'] = df['clean_text'].apply(translate_tweet)

In [28]:
df.head()

,created_at,hashtags,id_str,is_quote_status,lang,retweet_count,text,user_description,user_id_str,user_location,user_name,clean_text,emojis,translated_text
0,Sat May 18 11:58:18 +0000 2019,[],1129717847859576832,0.0,pl,0.0,@CarloCalenda Daje!,,758072107981017089,,Mario Bochicchio,Daje!,[],Gives!
1,Mon Mar 26 13:07:23 +0000 2018,[],978257103822426115,0.0,it,0.0,Scarica Fare Soldi Guadagna Denaro. E' un'app ...,,758072107981017089,,Mario Bochicchio,Scarica Fare Soldi Guadagna Denaro. E' un'app ...,[],Download Make Money Earn Money. It&#39;s a fre...
2,Tue Mar 06 12:59:09 +0000 2018,[],971007276755701760,0.0,it,0.0,@andreasso1951 @TraniCamilla Ahahahahahahahaha...,,758072107981017089,,Mario Bochicchio,"AhahahahahahahahahhahahahahaScelti perché ""dic...",[],Ahahahahahahahahahhahahahaha Chosen because &q...
3,Tue Nov 14 14:17:26 +0000 2017,[],930439536810844161,0.0,en,0.0,@Ballack Only the losers enjoy the defeats of ...,,758072107981017089,,Mario Bochicchio,Only the losers enjoy the defeats of those who...,[],Only the losers enjoy the defeats of those who...
4,Fri Jun 02 09:30:52 +0000 2017,[],870573420790108160,0.0,de,0.0,Zum 2. Jahrestag werden verschiedene Kampagnen...,,758072107981017089,,Mario Bochicchio,Zum 2. Jahrestag werden verschiedene Kampagnen...,[],For the 2nd anniversary different campaigns ar...
